## Baseline Models for Education Prediction

As a point of comparison, we also evaluate standard regression models trained directly on household-level features, without using simulated labels or clustering.

These models include:
- **Decision Tree**
- **Linear Regression**
- **Random Forest Regressor**
- **XGBoost**

Each baseline model is trained to predict the real percentage of higher education attainment (`Vyssi_vzdelani_real`) using only the observed household features available at the district level.

This allows us to evaluate whether the simulation-based approach improves over direct supervised learning when fine-grained labels are limited or incomplete.


In [1]:
# Baseline: Train Decision Tree on Kraj-level education, predict at Okres level using household stats

import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# Load data
krajske_vzdelani = pd.read_excel("./DATA/krajske_vzdelani_percentage_only.xlsx")
okresni_vzdelani = pd.read_excel("./DATA/vzdelani_with_percentage_only.xlsx")
domacnosti = pd.read_excel("./DATA/domacnosti.xlsx")

# Normalize text

def normalize_text(series):
    return (
        series.str.lower().str.replace(" ", "")
        .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
    )

# Normalize and clean
krajske_vzdelani.columns = ["Kraj", "Vyssi_vzdelani"]
krajske_vzdelani["Kraj"] = normalize_text(krajske_vzdelani["Kraj"])
okresni_vzdelani.columns = ["Okres", "Vyssi_vzdelani"]
okresni_vzdelani = okresni_vzdelani[~okresni_vzdelani["Okres"].str.contains("česká republika", case=False)]
okresni_vzdelani["Okres"] = normalize_text(okresni_vzdelani["Okres"])
domacnosti = domacnosti.rename(columns={"region": "Okres"})
domacnosti["Okres"] = normalize_text(domacnosti["Okres"].astype(str))

# Remove aggregate rows
domacnosti_okres = domacnosti[~domacnosti["Okres"].str.contains("kraj|ceskarepublika")].copy()

# Map Okres to Kraj manually
okres_to_kraj_map = {
    "hlavnimestopraha": "hlavnimestopraha",
    "benesov": "stredoceskykraj",
    "beroun": "stredoceskykraj",
    "kladno": "stredoceskykraj",
    "kolin": "stredoceskykraj",
    "kutnahora": "stredoceskykraj",
    "melnik": "stredoceskykraj",
    "mladaboleslav": "stredoceskykraj",
    "nymburk": "stredoceskykraj",
    "praha-vychod": "stredoceskykraj",
    "praha-zapad": "stredoceskykraj",
    "pribram": "stredoceskykraj",
    "rakovnik": "stredoceskykraj",
    "ceskebudejovice": "jihoceskykraj",
    "ceskykrumlov": "jihoceskykraj",
    "jindrichuvhradec": "jihoceskykraj",
    "pisek": "jihoceskykraj",
    "prachatice": "jihoceskykraj",
    "strakonice": "jihoceskykraj",
    "tabor": "jihoceskykraj",
    "domazlice": "plzenskykraj",
    "klatovy": "plzenskykraj",
    "plzen": "plzenskykraj",
    "plzen-jih": "plzenskykraj",
    "plzen-mesto": "plzenskykraj",
    "plzen-sever": "plzenskykraj",
    "rokycany": "plzenskykraj",
    "tachov": "plzenskykraj",
    "cheb": "karlovarskykraj",
    "karlovyvary": "karlovarskykraj",
    "sokolov": "karlovarskykraj",
    "decin": "usteckykraj",
    "chomutov": "usteckykraj",
    "litomerice": "usteckykraj",
    "louny": "usteckykraj",
    "most": "usteckykraj",
    "teplice": "usteckykraj",
    "usti": "usteckykraj",
    "ceskalipa": "libereckykraj",
    "jablonecnadnisou": "libereckykraj",
    "liberec": "libereckykraj",
    "semily": "libereckykraj",
    "hradeckralove": "kralovehradeckykraj",
    "jicin": "kralovehradeckykraj",
    "nachod": "kralovehradeckykraj",
    "rychnovnadkneznou": "kralovehradeckykraj",
    "trutnov": "kralovehradeckykraj",
    "chrudim": "pardubickykraj",
    "pardubice": "pardubickykraj",
    "svitavy": "pardubickykraj",
    "ustranadlabem": "pardubickykraj",
    "havlickuvbrod": "krajvysocina",
    "jihlava": "krajvysocina",
    "pelhrimov": "krajvysocina",
    "trebic": "krajvysocina",
    "zdarnadsazavou": "krajvysocina",
    "blansko": "jihomoravskykraj",
    "brno": "jihomoravskykraj",
    "brnomesto": "jihomoravskykraj",
    "brno-venkov": "jihomoravskykraj",
    "breclav": "jihomoravskykraj",
    "hodonin": "jihomoravskykraj",
    "vyskov": "jihomoravskykraj",
    "znojmo": "jihomoravskykraj",
    "jesenik": "olomouckykraj",
    "olomouc": "olomouckykraj",
    "prostejov": "olomouckykraj",
    "prerov": "olomouckykraj",
    "sumperk": "olomouckykraj",
    "bruntal": "moravskoslezskykraj",
    "frydekmistek": "moravskoslezskykraj",
    "karvina": "moravskoslezskykraj",
    "novyjicin": "moravskoslezskykraj",
    "opava": "moravskoslezskykraj",
    "ostrava": "moravskoslezskykraj",
    "ustinadlabem": "usteckykraj",
    "ustinadorlici": "pardubickykraj",
    "brno-mesto": "jihomoravskykraj",
    "kromeriz": "zlinskykraj",
    "uherskehradiste": "zlinskykraj",
    "vsetin": "zlinskykraj",
    "zlin": "zlinskykraj",
    "frydek-mistek": "moravskoslezskykraj",
    "ostrava-mesto": "moravskoslezskykraj",
}

domacnosti_okres["Kraj"] = domacnosti_okres["Okres"].map(okres_to_kraj_map)

# Merge for training at Kraj level
df_train = pd.merge(domacnosti_okres, krajske_vzdelani, on="Kraj", how="inner")

X_kraj = df_train.drop(columns=["Okres", "Kraj", "Vyssi_vzdelani"])
y_kraj = df_train["Vyssi_vzdelani"]

scaler = StandardScaler()
X_kraj_scaled = scaler.fit_transform(X_kraj)

# Train decision tree
model = DecisionTreeRegressor(random_state=42)
model.fit(X_kraj_scaled, y_kraj)

# Predict for Okres
X_okres = domacnosti_okres.drop(columns=["Okres", "Kraj"])
X_okres_scaled = scaler.transform(X_okres)
domacnosti_okres["Predicted_Education"] = model.predict(X_okres_scaled)

# Merge with real Okres data
df_eval = pd.merge(domacnosti_okres, okresni_vzdelani, on="Okres", how="left")
df_eval = df_eval.dropna(subset=["Vyssi_vzdelani"])

# Evaluate
mae = mean_absolute_error(df_eval["Vyssi_vzdelani"], df_eval["Predicted_Education"])
print(f"🌳 Decision Tree MAE (Kraj-trained → Okres prediction): {mae:.4f}")

# Save output
df_output = df_eval[["Okres", "Vyssi_vzdelani", "Predicted_Education"]]

🌳 Decision Tree MAE (Kraj-trained → Okres prediction): 2.9372


In [2]:
# Baseline: Train on Kraj-level education, predict at Okres-level using household features

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# Load files
krajske_vzdelani = pd.read_excel("./DATA/krajske_vzdelani_percentage_only.xlsx")
okresni_vzdelani = pd.read_excel("./DATA/vzdelani_with_percentage_only.xlsx")
domacnosti = pd.read_excel("./DATA/domacnosti.xlsx")

# Normalize function
def normalize_text(series):
    return (
        series.str.lower().str.replace(" ", "")
        .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
    )

# Normalize names
krajske_vzdelani.columns = ["Kraj", "Vyssi_vzdelani"]
krajske_vzdelani["Kraj"] = normalize_text(krajske_vzdelani["Kraj"])
okresni_vzdelani.columns = ["Okres", "Vyssi_vzdelani"]
okresni_vzdelani = okresni_vzdelani[~okresni_vzdelani["Okres"].str.contains("česká republika", case=False)]
okresni_vzdelani["Okres"] = normalize_text(okresni_vzdelani["Okres"])
domacnosti = domacnosti.rename(columns={"region": "Okres"})
domacnosti["Okres"] = normalize_text(domacnosti["Okres"].astype(str))

# Remove aggregate rows
domacnosti_okres = domacnosti[~domacnosti["Okres"].str.contains("kraj|ceskarepublika")].copy()

# Map Okres to Kraj manually
okres_to_kraj_map = {
    "hlavnimestopraha": "hlavnimestopraha",
    "benesov": "stredoceskykraj",
    "beroun": "stredoceskykraj",
    "kladno": "stredoceskykraj",
    "kolin": "stredoceskykraj",
    "kutnahora": "stredoceskykraj",
    "melnik": "stredoceskykraj",
    "mladaboleslav": "stredoceskykraj",
    "nymburk": "stredoceskykraj",
    "praha-vychod": "stredoceskykraj",
    "praha-zapad": "stredoceskykraj",
    "pribram": "stredoceskykraj",
    "rakovnik": "stredoceskykraj",
    "ceskebudejovice": "jihoceskykraj",
    "ceskykrumlov": "jihoceskykraj",
    "jindrichuvhradec": "jihoceskykraj",
    "pisek": "jihoceskykraj",
    "prachatice": "jihoceskykraj",
    "strakonice": "jihoceskykraj",
    "tabor": "jihoceskykraj",
    "domazlice": "plzenskykraj",
    "klatovy": "plzenskykraj",
    "plzen": "plzenskykraj",
    "plzen-jih": "plzenskykraj",
    "plzen-mesto": "plzenskykraj",
    "plzen-sever": "plzenskykraj",
    "rokycany": "plzenskykraj",
    "tachov": "plzenskykraj",
    "cheb": "karlovarskykraj",
    "karlovyvary": "karlovarskykraj",
    "sokolov": "karlovarskykraj",
    "decin": "usteckykraj",
    "chomutov": "usteckykraj",
    "litomerice": "usteckykraj",
    "louny": "usteckykraj",
    "most": "usteckykraj",
    "teplice": "usteckykraj",
    "usti": "usteckykraj",
    "ceskalipa": "libereckykraj",
    "jablonecnadnisou": "libereckykraj",
    "liberec": "libereckykraj",
    "semily": "libereckykraj",
    "hradeckralove": "kralovehradeckykraj",
    "jicin": "kralovehradeckykraj",
    "nachod": "kralovehradeckykraj",
    "rychnovnadkneznou": "kralovehradeckykraj",
    "trutnov": "kralovehradeckykraj",
    "chrudim": "pardubickykraj",
    "pardubice": "pardubickykraj",
    "svitavy": "pardubickykraj",
    "ustranadlabem": "pardubickykraj",
    "havlickuvbrod": "krajvysocina",
    "jihlava": "krajvysocina",
    "pelhrimov": "krajvysocina",
    "trebic": "krajvysocina",
    "zdarnadsazavou": "krajvysocina",
    "blansko": "jihomoravskykraj",
    "brno": "jihomoravskykraj",
    "brnomesto": "jihomoravskykraj",
    "brno-venkov": "jihomoravskykraj",
    "breclav": "jihomoravskykraj",
    "hodonin": "jihomoravskykraj",
    "vyskov": "jihomoravskykraj",
    "znojmo": "jihomoravskykraj",
    "jesenik": "olomouckykraj",
    "olomouc": "olomouckykraj",
    "prostejov": "olomouckykraj",
    "prerov": "olomouckykraj",
    "sumperk": "olomouckykraj",
    "bruntal": "moravskoslezskykraj",
    "frydekmistek": "moravskoslezskykraj",
    "karvina": "moravskoslezskykraj",
    "novyjicin": "moravskoslezskykraj",
    "opava": "moravskoslezskykraj",
    "ostrava": "moravskoslezskykraj",
    "ustinadlabem": "usteckykraj",
    "ustinadorlici": "pardubickykraj",
    "brno-mesto": "jihomoravskykraj",
    "kromeriz": "zlinskykraj",
    "uherskehradiste": "zlinskykraj",
    "vsetin": "zlinskykraj",
    "zlin": "zlinskykraj",
    "frydek-mistek": "moravskoslezskykraj",
    "ostrava-mesto": "moravskoslezskykraj",
}
domacnosti_okres["Kraj"] = domacnosti_okres["Okres"].map(okres_to_kraj_map)

# Group to Kraj level for training
df_kraj_train = domacnosti_okres.groupby("Kraj").mean(numeric_only=True).reset_index()
df_kraj_train = pd.merge(df_kraj_train, krajske_vzdelani, on="Kraj", how="inner")

X_kraj = df_kraj_train.drop(columns=["Kraj", "Vyssi_vzdelani"])
y_kraj = df_kraj_train["Vyssi_vzdelani"]

scaler = StandardScaler()
X_kraj_scaled = scaler.fit_transform(X_kraj)

# Train model
model = LinearRegression()
model.fit(X_kraj_scaled, y_kraj)

# Prepare Okres-level prediction
X_okres = domacnosti_okres.drop(columns=["Okres", "Kraj"])
X_okres_scaled = scaler.transform(X_okres)
domacnosti_okres["LR_Predicted"] = model.predict(X_okres_scaled)

# Merge with real education for evaluation
df_eval = pd.merge(domacnosti_okres, okresni_vzdelani, on="Okres", how="left")
df_eval = df_eval.dropna(subset=["Vyssi_vzdelani"])

# Compute MAE
mae = mean_absolute_error(df_eval["Vyssi_vzdelani"], df_eval["LR_Predicted"])
print(f"📉 MAE (Kraj-trained Linear Regression → Okres prediction): {mae:.4f}")

# Save output
df_output = df_eval[["Okres", "Vyssi_vzdelani", "LR_Predicted"]]

📉 MAE (Kraj-trained Linear Regression → Okres prediction): 2.0845


In [3]:
# Baseline: Train Random Forest on Kraj-level education, predict at Okres level using household stats

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

# Load data
krajske_vzdelani = pd.read_excel("./DATA/krajske_vzdelani_percentage_only.xlsx")
okresni_vzdelani = pd.read_excel("./DATA/vzdelani_with_percentage_only.xlsx")
domacnosti = pd.read_excel("./DATA/domacnosti.xlsx")

# Normalize function
def normalize_text(series):
    return (
        series.str.lower().str.replace(" ", "")
        .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
    )

# Normalize names
krajske_vzdelani.columns = ["Kraj", "Vyssi_vzdelani"]
krajske_vzdelani["Kraj"] = normalize_text(krajske_vzdelani["Kraj"])
okresni_vzdelani.columns = ["Okres", "Vyssi_vzdelani"]
okresni_vzdelani = okresni_vzdelani[~okresni_vzdelani["Okres"].str.contains("česká republika", case=False)]
okresni_vzdelani["Okres"] = normalize_text(okresni_vzdelani["Okres"])
domacnosti = domacnosti.rename(columns={"region": "Okres"})
domacnosti["Okres"] = normalize_text(domacnosti["Okres"].astype(str))

# Remove aggregate rows
domacnosti_okres = domacnosti[~domacnosti["Okres"].str.contains("kraj|ceskarepublika")].copy()

# Okres to Kraj mapping
okres_to_kraj_map = {
    "hlavnimestopraha": "hlavnimestopraha",
    "benesov": "stredoceskykraj",
    "beroun": "stredoceskykraj",
    "kladno": "stredoceskykraj",
    "kolin": "stredoceskykraj",
    "kutnahora": "stredoceskykraj",
    "melnik": "stredoceskykraj",
    "mladaboleslav": "stredoceskykraj",
    "nymburk": "stredoceskykraj",
    "praha-vychod": "stredoceskykraj",
    "praha-zapad": "stredoceskykraj",
    "pribram": "stredoceskykraj",
    "rakovnik": "stredoceskykraj",
    "ceskebudejovice": "jihoceskykraj",
    "ceskykrumlov": "jihoceskykraj",
    "jindrichuvhradec": "jihoceskykraj",
    "pisek": "jihoceskykraj",
    "prachatice": "jihoceskykraj",
    "strakonice": "jihoceskykraj",
    "tabor": "jihoceskykraj",
    "domazlice": "plzenskykraj",
    "klatovy": "plzenskykraj",
    "plzen": "plzenskykraj",
    "plzen-jih": "plzenskykraj",
    "plzen-mesto": "plzenskykraj",
    "plzen-sever": "plzenskykraj",
    "rokycany": "plzenskykraj",
    "tachov": "plzenskykraj",
    "cheb": "karlovarskykraj",
    "karlovyvary": "karlovarskykraj",
    "sokolov": "karlovarskykraj",
    "decin": "usteckykraj",
    "chomutov": "usteckykraj",
    "litomerice": "usteckykraj",
    "louny": "usteckykraj",
    "most": "usteckykraj",
    "teplice": "usteckykraj",
    "usti": "usteckykraj",
    "ceskalipa": "libereckykraj",
    "jablonecnadnisou": "libereckykraj",
    "liberec": "libereckykraj",
    "semily": "libereckykraj",
    "hradeckralove": "kralovehradeckykraj",
    "jicin": "kralovehradeckykraj",
    "nachod": "kralovehradeckykraj",
    "rychnovnadkneznou": "kralovehradeckykraj",
    "trutnov": "kralovehradeckykraj",
    "chrudim": "pardubickykraj",
    "pardubice": "pardubickykraj",
    "svitavy": "pardubickykraj",
    "ustranadlabem": "pardubickykraj",
    "havlickuvbrod": "krajvysocina",
    "jihlava": "krajvysocina",
    "pelhrimov": "krajvysocina",
    "trebic": "krajvysocina",
    "zdarnadsazavou": "krajvysocina",
    "blansko": "jihomoravskykraj",
    "brno": "jihomoravskykraj",
    "brnomesto": "jihomoravskykraj",
    "brno-venkov": "jihomoravskykraj",
    "breclav": "jihomoravskykraj",
    "hodonin": "jihomoravskykraj",
    "vyskov": "jihomoravskykraj",
    "znojmo": "jihomoravskykraj",
    "jesenik": "olomouckykraj",
    "olomouc": "olomouckykraj",
    "prostejov": "olomouckykraj",
    "prerov": "olomouckykraj",
    "sumperk": "olomouckykraj",
    "bruntal": "moravskoslezskykraj",
    "frydekmistek": "moravskoslezskykraj",
    "karvina": "moravskoslezskykraj",
    "novyjicin": "moravskoslezskykraj",
    "opava": "moravskoslezskykraj",
    "ostrava": "moravskoslezskykraj",
    "ustinadlabem": "usteckykraj",
    "ustinadorlici": "pardubickykraj",
    "brno-mesto": "jihomoravskykraj",
    "kromeriz": "zlinskykraj",
    "uherskehradiste": "zlinskykraj",
    "vsetin": "zlinskykraj",
    "zlin": "zlinskykraj",
    "frydek-mistek": "moravskoslezskykraj",
    "ostrava-mesto": "moravskoslezskykraj",
}
domacnosti_okres["Kraj"] = domacnosti_okres["Okres"].map(okres_to_kraj_map)

# Merge household features with Kraj-level target
df_train = pd.merge(domacnosti_okres, krajske_vzdelani, on="Kraj", how="inner")

# Prepare training features
X_kraj = df_train.drop(columns=["Okres", "Kraj", "Vyssi_vzdelani"])
y_kraj = df_train["Vyssi_vzdelani"]

scaler = StandardScaler()
X_kraj_scaled = scaler.fit_transform(X_kraj)

# Train model
model = RandomForestRegressor(random_state=42)
model.fit(X_kraj_scaled, y_kraj)

# Predict for Okres-level
X_okres = domacnosti_okres.drop(columns=["Okres", "Kraj"])
X_okres_scaled = scaler.transform(X_okres)
domacnosti_okres["Predicted_Education"] = model.predict(X_okres_scaled)

# Merge with real Okres-level data
df_eval = pd.merge(domacnosti_okres, okresni_vzdelani, on="Okres", how="left")
df_eval = df_eval.dropna(subset=["Vyssi_vzdelani"])

# Evaluate
mae = mean_absolute_error(df_eval["Vyssi_vzdelani"], df_eval["Predicted_Education"])
print(f"🌲 Random Forest MAE (Kraj-trained → Okres prediction): {mae:.4f}")

# Save output
df_output = df_eval[["Okres", "Vyssi_vzdelani", "Predicted_Education"]]

🌲 Random Forest MAE (Kraj-trained → Okres prediction): 2.9146


In [4]:
# Baseline: Train XGBoost on Kraj-level education, predict at Okres level using household stats

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Load data
krajske_vzdelani = pd.read_excel("./DATA/krajske_vzdelani_percentage_only.xlsx")
okresni_vzdelani = pd.read_excel("./DATA/vzdelani_with_percentage_only.xlsx")
domacnosti = pd.read_excel("./DATA/domacnosti.xlsx")

# Normalize function
def normalize_text(series):
    return (
        series.str.lower().str.replace(" ", "")
        .str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf-8")
    )

# Normalize and clean
krajske_vzdelani.columns = ["Kraj", "Vyssi_vzdelani"]
krajske_vzdelani["Kraj"] = normalize_text(krajske_vzdelani["Kraj"])
okresni_vzdelani.columns = ["Okres", "Vyssi_vzdelani"]
okresni_vzdelani = okresni_vzdelani[~okresni_vzdelani["Okres"].str.contains("česká republika", case=False)]
okresni_vzdelani["Okres"] = normalize_text(okresni_vzdelani["Okres"])
domacnosti = domacnosti.rename(columns={"region": "Okres"})
domacnosti["Okres"] = normalize_text(domacnosti["Okres"].astype(str))

# Filter only real Okres rows
domacnosti_okres = domacnosti[~domacnosti["Okres"].str.contains("kraj|ceskarepublika")].copy()

# Map Okres to Kraj manually
okres_to_kraj_map = {
    "hlavnimestopraha": "hlavnimestopraha",
    "benesov": "stredoceskykraj",
    "beroun": "stredoceskykraj",
    "kladno": "stredoceskykraj",
    "kolin": "stredoceskykraj",
    "kutnahora": "stredoceskykraj",
    "melnik": "stredoceskykraj",
    "mladaboleslav": "stredoceskykraj",
    "nymburk": "stredoceskykraj",
    "praha-vychod": "stredoceskykraj",
    "praha-zapad": "stredoceskykraj",
    "pribram": "stredoceskykraj",
    "rakovnik": "stredoceskykraj",
    "ceskebudejovice": "jihoceskykraj",
    "ceskykrumlov": "jihoceskykraj",
    "jindrichuvhradec": "jihoceskykraj",
    "pisek": "jihoceskykraj",
    "prachatice": "jihoceskykraj",
    "strakonice": "jihoceskykraj",
    "tabor": "jihoceskykraj",
    "domazlice": "plzenskykraj",
    "klatovy": "plzenskykraj",
    "plzen": "plzenskykraj",
    "plzen-jih": "plzenskykraj",
    "plzen-mesto": "plzenskykraj",
    "plzen-sever": "plzenskykraj",
    "rokycany": "plzenskykraj",
    "tachov": "plzenskykraj",
    "cheb": "karlovarskykraj",
    "karlovyvary": "karlovarskykraj",
    "sokolov": "karlovarskykraj",
    "decin": "usteckykraj",
    "chomutov": "usteckykraj",
    "litomerice": "usteckykraj",
    "louny": "usteckykraj",
    "most": "usteckykraj",
    "teplice": "usteckykraj",
    "usti": "usteckykraj",
    "ceskalipa": "libereckykraj",
    "jablonecnadnisou": "libereckykraj",
    "liberec": "libereckykraj",
    "semily": "libereckykraj",
    "hradeckralove": "kralovehradeckykraj",
    "jicin": "kralovehradeckykraj",
    "nachod": "kralovehradeckykraj",
    "rychnovnadkneznou": "kralovehradeckykraj",
    "trutnov": "kralovehradeckykraj",
    "chrudim": "pardubickykraj",
    "pardubice": "pardubickykraj",
    "svitavy": "pardubickykraj",
    "ustranadlabem": "pardubickykraj",
    "havlickuvbrod": "krajvysocina",
    "jihlava": "krajvysocina",
    "pelhrimov": "krajvysocina",
    "trebic": "krajvysocina",
    "zdarnadsazavou": "krajvysocina",
    "blansko": "jihomoravskykraj",
    "brno": "jihomoravskykraj",
    "brnomesto": "jihomoravskykraj",
    "brno-venkov": "jihomoravskykraj",
    "breclav": "jihomoravskykraj",
    "hodonin": "jihomoravskykraj",
    "vyskov": "jihomoravskykraj",
    "znojmo": "jihomoravskykraj",
    "jesenik": "olomouckykraj",
    "olomouc": "olomouckykraj",
    "prostejov": "olomouckykraj",
    "prerov": "olomouckykraj",
    "sumperk": "olomouckykraj",
    "bruntal": "moravskoslezskykraj",
    "frydekmistek": "moravskoslezskykraj",
    "karvina": "moravskoslezskykraj",
    "novyjicin": "moravskoslezskykraj",
    "opava": "moravskoslezskykraj",
    "ostrava": "moravskoslezskykraj",
    "ustinadlabem": "usteckykraj",
    "ustinadorlici": "pardubickykraj",
    "brno-mesto": "jihomoravskykraj",
    "kromeriz": "zlinskykraj",
    "uherskehradiste": "zlinskykraj",
    "vsetin": "zlinskykraj",
    "zlin": "zlinskykraj",
    "frydek-mistek": "moravskoslezskykraj",
    "ostrava-mesto": "moravskoslezskykraj",
}
domacnosti_okres["Kraj"] = domacnosti_okres["Okres"].map(okres_to_kraj_map)

# Merge for training
kraj_train = pd.merge(domacnosti_okres, krajske_vzdelani, on="Kraj", how="inner")
X_kraj = kraj_train.drop(columns=["Okres", "Kraj", "Vyssi_vzdelani"])
y_kraj = kraj_train["Vyssi_vzdelani"]

scaler = StandardScaler()
X_kraj_scaled = scaler.fit_transform(X_kraj)

# Train model
model = XGBRegressor(random_state=42)
model.fit(X_kraj_scaled, y_kraj)

# Predict for Okres
X_okres = domacnosti_okres.drop(columns=["Okres", "Kraj"])
X_okres_scaled = scaler.transform(X_okres)
domacnosti_okres["Predicted_Education"] = model.predict(X_okres_scaled)

# Merge with ground truth
df_eval = pd.merge(domacnosti_okres, okresni_vzdelani, on="Okres", how="left")
df_eval = df_eval.dropna(subset=["Vyssi_vzdelani"])

# MAE
mae = mean_absolute_error(df_eval["Vyssi_vzdelani"], df_eval["Predicted_Education"])
print(f"⚡ XGBoost MAE (Kraj-trained → Okres prediction): {mae:.4f}")


⚡ XGBoost MAE (Kraj-trained → Okres prediction): 2.9370
